In [4]:
import os
%pwd

'd:\\coding\\mlops\\end_to_end_nlp_project\\research'

In [5]:
os.chdir("../")
%pwd

'd:\\coding\\mlops\\end_to_end_nlp_project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [7]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config


In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

d:\coding\mlops\end_to_end_nlp_project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-02-02 21:19:25,227: INFO: config: PyTorch version 2.5.1 available.]


In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [11]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

   ---------------------------------------- 0.0/336.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/336.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/336.6 kB ? eta -:--:--
   ---- ----------------------------------- 41.0/336.6 kB ? eta -:--:--
   ------- -------------------------------- 61.4/336.6 kB 1.1 MB/s eta 0:00:01
   ------- -------------------------------- 61.4/336.6 kB 1.1 MB/s eta 0:00:01
   -------- ------------------------------ 71.7/336.6 kB 491.5 kB/s eta 0:00:01
   ---------- ---------------------------- 92.2/336.6 kB 438.1 kB/s eta 0:00:01
   ---------- ---------------------------- 92.2/336.6 kB 438.1 kB/s eta 0:00:01
   ------------ ------------------------- 112.6/336.6 kB 409.6 kB/s eta 0:00:01
   ------------ ------------------------- 112.6/336.6 kB 409.6 kB/s eta 0:00:01
   ------------ ------------------------- 112.6/336.6 kB 409.6 kB/s eta 0:00:01
   ------------ ------------------------- 112.6/336.6 kB 409.6 kB/s eta 0:00:


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Found existing installation: transformers 4.48.1
Uninstalling transformers-4.48.1:
  Successfully uninstalled transformers-4.48.1
Found existing installation: accelerate 1.3.0
Uninstalling accelerate-1.3.0:
  Successfully uninstalled accelerate-1.3.0
     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ---------------------------------------- 44.4/44.4 kB 2.1 MB/s eta 0:00:00
  Using cached accelerate-1.3.0-py3-none-any.whl.metadata (19 kB)
   ------------------------


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-02-02 21:24:00,169: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-02 21:24:00,179: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-02 21:24:00,180: INFO: common: created directory at: artifacts]
[2025-02-02 21:24:00,181: INFO: common: created directory at: artifacts/model_trainer]


ChunkedEncodingError: ('Connection broken: IncompleteRead(98271767 bytes read, 2177056116 more expected)', IncompleteRead(98271767 bytes read, 2177056116 more expected))